In [1]:
from datasets import load_dataset
import re


In [2]:
from kaggle_secrets import UserSecretsClient
huggingface_token = UserSecretsClient().get_secret("huggingface_token")

from huggingface_hub import login

login(token=huggingface_token)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
dataset = load_dataset("vishnun0027/openassistant-guanaco")
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 10364
    })
})

In [4]:
dataset= dataset['train']
dataset

Dataset({
    features: ['text'],
    num_rows: 10364
})

In [5]:
dataset[0]

{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining po

In [6]:
# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}




In [7]:
# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

Map:   0%|          | 0/10364 [00:00<?, ? examples/s]

In [9]:
transformed_dataset

Dataset({
    features: ['text'],
    num_rows: 10364
})

In [8]:
transformed_dataset[0]

{'text': '<s>[INST] Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research. [/INST] "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, le

In [10]:
transformed_dataset.push_to_hub("vishnun0027/guanaco-llama2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/vishnun0027/guanaco-llama2/commit/ffb6beba79c342948538d0a19611c61cff535c46', commit_message='Upload dataset', commit_description='', oid='ffb6beba79c342948538d0a19611c61cff535c46', pr_url=None, pr_revision=None, pr_num=None)